# Playing with Reinforcement Learning Neural Networks

In [80]:
import numpy as np
import random
import itertools
import math
import time
from typing import Optional, Tuple, cast, List, Dict

## General auxiliary functions

In [2]:
def get_combinations_two_etas_without_repeats_from_etas(angles_etas: List[float]) -> List[Tuple[float, float]]:
    """ from a given list of attenuations factors create a
        list of all combinatorial pairs of possible etas
        without repeats
        For us it is the same testing first eta 0.1 and second eta 0.2
        than first eta 0.2 and second eta 0.1
        Though, we will always put the greater value as the first pair element
    """
    # when there is only one element, we add the same element
    if len(angles_etas) == 1:
        angles_etas.append(angles_etas[0])
    # get combinations of two etas without repeats
    eta_pairs = list(itertools.combinations(angles_etas, 2))

    return reorder_pairs(eta_pairs)

In [3]:
def reorder_pairs(pairs: List[Tuple[float, float]]) -> List[Tuple[float, float]]:
    """ reorder received pairs setting first the element of the tuple
        as greater or equal de second one
    """
    reordered_pairs = pairs
    for idx, pair in enumerate(pairs):
        reordered_pairs[idx] = reorder_pair(pair)
    return reordered_pairs

In [4]:
def reorder_pair(pair: Tuple[float, float]) -> Tuple[float, float]:
    if pair[0] < pair[1]:
        return (pair[1], pair[0])
    return pair

## Setting the input variables (features)

In [89]:
def get_entangled_ys(num_etas_to_be_split: int)-> Tuple[List[float], Tuple[int, int, int]]:
    etas = np.append(np.arange(0, np.pi/2, np.pi/2/num_etas_to_be_split), np.pi/2)
    eta_pairs = get_combinations_two_etas_without_repeats_from_etas(etas)
    gammas = [np.cos(eta_pair[1]) + np.cos(eta_pair[0]) for eta_pair in eta_pairs]
    gammas_etas = [(eta_pair[0], eta_pair[1], gammas[idx]) for idx, eta_pair in enumerate(eta_pairs)]
    entangled_gammas = [gamma for gamma in gammas if gamma <= 1]
    entangled_etas = [(eta_pair[0], eta_pair[1]) for idx, eta_pair in enumerate(eta_pairs) if gammas[idx] <= 1]
    theoretical_ys = [(gamma-1)/(gamma-2) for gamma in gammas] 
    entangled_ys = [theoretical_y for theoretical_y in theoretical_ys if theoretical_y > 0]
    entangled_ys.sort()
    entangled_ys.reverse()
    return (entangled_ys, (len(entangled_ys), max(entangled_ys), min(entangled_ys)), entangled_gammas, entangled_etas)

In [73]:
get_entangled_ys(20)[1]

(59, 0.47958432850599036, 0.003073192639590261)

In [63]:
def get_etas_from_entangled_ys(ty_min:float = 0, ty_max:float = 0.5):
    gamma_min = (1-2*ty_min)/(1-ty_min)
    gamma_max = (1-2*ty_max)/(1-ty_max)
    return (gamma_min, gamma_max)

In [64]:
get_etas_from_entangled_ys(0, 0.5)

(1.0, 0.0)

In [75]:
len(get_entangled_ys(20)[2])

60

In [83]:
lst = [(math.degrees(elem[0]), math.degrees(elem[1])) for elem in get_entangled_ys(20)[3]]
lst.sort()
lst

[(63.0, 58.5),
 (67.5, 54.0),
 (67.5, 58.5),
 (67.5, 63.0),
 (72.0, 49.5),
 (72.0, 54.0),
 (72.0, 58.5),
 (72.0, 63.0),
 (72.0, 67.5),
 (76.5, 40.5),
 (76.5, 45.0),
 (76.5, 49.5),
 (76.5, 54.0),
 (76.5, 58.5),
 (76.5, 63.0),
 (76.5, 67.5),
 (76.5, 72.0),
 (81.0, 36.0),
 (81.0, 40.5),
 (81.0, 45.0),
 (81.0, 49.5),
 (81.0, 54.0),
 (81.0, 58.5),
 (81.0, 63.0),
 (81.0, 67.5),
 (81.0, 72.0),
 (81.0, 76.5),
 (85.5, 27.0),
 (85.5, 31.5),
 (85.5, 36.0),
 (85.5, 40.5),
 (85.5, 45.0),
 (85.5, 49.5),
 (85.5, 54.0),
 (85.5, 58.5),
 (85.5, 63.0),
 (85.5, 67.5),
 (85.5, 72.0),
 (85.5, 76.5),
 (85.5, 81.0),
 (90.0, 0.0),
 (90.0, 4.5),
 (90.0, 9.0),
 (90.0, 13.5),
 (90.0, 18.0),
 (90.0, 22.5),
 (90.0, 27.0),
 (90.0, 31.5),
 (90.0, 36.0),
 (90.0, 40.5),
 (90.0, 45.0),
 (90.0, 49.5),
 (90.0, 54.0),
 (90.0, 58.5),
 (90.0, 63.0),
 (90.0, 67.5),
 (90.0, 72.0),
 (90.0, 76.5),
 (90.0, 81.0),
 (90.0, 85.5)]

In [90]:
len(get_entangled_ys(20)[3])

60

In [94]:
lst2 = get_entangled_ys(20)[3]
lst2.sort()
lst2

[(1.0995574287564276, 1.0210176124166828),
 (1.1780972450961724, 0.9424777960769379),
 (1.1780972450961724, 1.0210176124166828),
 (1.1780972450961724, 1.0995574287564276),
 (1.2566370614359172, 0.8639379797371931),
 (1.2566370614359172, 0.9424777960769379),
 (1.2566370614359172, 1.0210176124166828),
 (1.2566370614359172, 1.0995574287564276),
 (1.2566370614359172, 1.1780972450961724),
 (1.335176877775662, 0.7068583470577035),
 (1.335176877775662, 0.7853981633974483),
 (1.335176877775662, 0.8639379797371931),
 (1.335176877775662, 0.9424777960769379),
 (1.335176877775662, 1.0210176124166828),
 (1.335176877775662, 1.0995574287564276),
 (1.335176877775662, 1.1780972450961724),
 (1.335176877775662, 1.2566370614359172),
 (1.413716694115407, 0.6283185307179586),
 (1.413716694115407, 0.7068583470577035),
 (1.413716694115407, 0.7853981633974483),
 (1.413716694115407, 0.8639379797371931),
 (1.413716694115407, 0.9424777960769379),
 (1.413716694115407, 1.0210176124166828),
 (1.413716694115407, 1.09

In [39]:
features = { 'amplitude_probability': (0, 1),
            'rx_angle': (0, 2*np.pi),
            'ry_angle': (0, 2*np.pi),
            'measurement': ('00', '01', '10', '11'),
            'entangled_ys': get_entangled_ys(num_etas_to_be_split=20)[0]}

In [56]:
0.5**2

0.25

In [59]:
np.sqrt(0.8)

0.8944271909999159

In [60]:
features

{'amplitude_probability': (0, 1),
 'rx_angle': (0, 6.283185307179586),
 'ry_angle': (0, 6.283185307179586),
 'measurement': ('00', '01', '10', '11'),
 'entangled_ys': [0.47958432850599036,
  0.457572849439376,
  0.43392636743875274,
  0.4334619274092364,
  0.4086280011842216,
  0.4076164671619115,
  0.38169185921198207,
  0.3798541566656544,
  0.37892834463751757,
  0.35317344438599374,
  0.35016723884069223,
  0.34834254273970594,
  0.32318170654923994,
  0.31859239934012057,
  0.3154820651003284,
  0.313911372617308,
  0.2918923969436534,
  0.2852253489932173,
  0.28035552262097296,
  0.2773894006398492,
  0.25956198962683025,
  0.25023747901970733,
  0.2430361045091931,
  0.23813246738612076,
  0.2356490665908845,
  0.22654091966098633,
  0.21389386369950902,
  0.20368235287467223,
  0.19617771017987584,
  0.19328427513443577,
  0.1915876650488305,
  0.17657122688613025,
  0.16256146384514042,
  0.1603574565909281,
  0.15165764084192068,
  0.1441905728569389,
  0.140395777592129,
  

In [61]:
np.pi/2

1.5707963267948966